In [2]:
import torch 
import torch.nn as nn 
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.autograd import Variable
import numpy as np
import matplotlib.pyplot as plt
import time
from logger import Logger


In [3]:
data_transforms = transforms.Compose([
    transforms.RandomSizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
])

data_dir = './data'
batch_size = 16
learning_rate = 0.001
total_step = 20000
image_datasets = datasets.ImageFolder(data_dir, data_transforms)
dataloader = torch.utils.data.DataLoader(image_datasets, batch_size=batch_size, shuffle=True, num_workers=8)
class_names = image_datasets.classes

In [4]:
class VGG_16(nn.Module):
    def __init__(self):
        super(VGG_16, self).__init__()
        self.layers1 = nn.Sequential(nn.Conv2d(3, 64, kernel_size=3, padding=1),
                                     nn.BatchNorm2d(64),
                                     nn.ReLU(),
                                     nn.Conv2d(64, 64, kernel_size=3, padding=1),
                                     nn.BatchNorm2d(64),
                                     nn.ReLU(),
                                     nn.MaxPool2d(2))
        
        self.layers2 = nn.Sequential(nn.Conv2d(64, 128, kernel_size=3, padding=1),
                                     nn.BatchNorm2d(128),
                                     nn.ReLU(),
                                     nn.Conv2d(128, 128, kernel_size=3, padding=1),
                                     nn.BatchNorm2d(128),
                                     nn.ReLU(),
                                     nn.MaxPool2d(2))
        
        self.layers3 = nn.Sequential(nn.Conv2d(128, 256, kernel_size=3, padding=1),
                                     nn.BatchNorm2d(256),
                                     nn.ReLU(),
                                     nn.Conv2d(256, 256, kernel_size=3, padding=1),
                                     nn.BatchNorm2d(256),
                                     nn.ReLU(),
                                     nn.MaxPool2d(2))
        
        self.layers4 = nn.Sequential(nn.Conv2d(256, 512, kernel_size=3, padding=1),
                                     nn.BatchNorm2d(512),
                                     nn.ReLU(),
                                     nn.Conv2d(512, 512, kernel_size=3, padding=1),
                                     nn.BatchNorm2d(512),
                                     nn.ReLU(),
                                     nn.Conv2d(512, 512, kernel_size=3, padding=1),
                                     nn.BatchNorm2d(512),
                                     nn.ReLU(),
                                     nn.MaxPool2d(2))
        
        self.layers5 = nn.Sequential(nn.Conv2d(512, 512, kernel_size=3, padding=1),
                                     nn.BatchNorm2d(512),
                                     nn.ReLU(),
                                     nn.Conv2d(512, 512, kernel_size=3, padding=1),
                                     nn.BatchNorm2d(512),
                                     nn.ReLU(),
                                     nn.Conv2d(512, 512, kernel_size=3, padding=1),
                                     nn.BatchNorm2d(512),
                                     nn.ReLU(),
                                     nn.MaxPool2d(2))
        
        self.fc1 = nn.Linear(7*7*512, 1024)
        self.fc2 = nn.Linear(1024, 1024)
        self.fc3 = nn.Linear(1024, 2)
    def forward(self, x):
        out = self.layers1(x)
        out = self.layers2(out)
        out = self.layers3(out)
        out = self.layers4(out)
        out = self.layers5(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.fc2(out)
        out = self.fc3(out)
        return out

In [4]:
cnn = VGG_16()
cnn.cuda()

VGG_16 (
  (layers1): Sequential (
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    (2): ReLU ()
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    (5): ReLU ()
    (6): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
  )
  (layers2): Sequential (
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
    (2): ReLU ()
    (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
    (5): ReLU ()
    (6): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
  )
  (layers3): Sequential (
    (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
    (2): R

In [5]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cnn.parameters(), lr=learning_rate)
logger = Logger('./logs_vgg_bn')

data_iter = iter(dataloader)
iter_per_epoch = len(dataloader)

In [6]:
for step in range(total_step):
    if (step+1)%iter_per_epoch:
        data_iter = iter(dataloader)
    images, labels = next(data_iter)
    images, labels = Variable(images).cuda(), Variable(labels).cuda()
    optimizer.zero_grad()
    outputs = cnn(images)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    
    _, argmax = torch.max(outputs, 1)
    accuracy = (labels == argmax.squeeze()).float().mean()
    if (step+1) % 100 == 0:
        print ('Step [%d/%d], Loss: %.4f, Acc: %.4f' 
               %(step+1, total_step, loss.data[0], accuracy.data[0]))

        #============ TensorBoard logging ============#
        # (1) Log the scalar values
        info = {
            'loss': loss.data[0],
            'accuracy': accuracy.data[0]
        }

        for tag, value in info.items():
            logger.scalar_summary(tag, value, step+1)

Step [100/20000], Loss: 0.8800, Acc: 0.8750
Step [200/20000], Loss: 0.6777, Acc: 0.5625
Step [300/20000], Loss: 0.6200, Acc: 0.5625
Step [400/20000], Loss: 0.7457, Acc: 0.4375
Step [500/20000], Loss: 0.9846, Acc: 0.5000
Step [600/20000], Loss: 0.8107, Acc: 0.3750
Step [700/20000], Loss: 0.8996, Acc: 0.5000
Step [800/20000], Loss: 0.5526, Acc: 0.8125
Step [900/20000], Loss: 0.6066, Acc: 0.6250
Step [1000/20000], Loss: 0.7383, Acc: 0.6875
Step [1100/20000], Loss: 0.6460, Acc: 0.6250
Step [1200/20000], Loss: 0.5830, Acc: 0.6250
Step [1300/20000], Loss: 0.7629, Acc: 0.4375
Step [1400/20000], Loss: 0.8527, Acc: 0.7500
Step [1500/20000], Loss: 0.6066, Acc: 0.6875
Step [1600/20000], Loss: 0.7787, Acc: 0.4375
Step [1700/20000], Loss: 0.6014, Acc: 0.7500
Step [1800/20000], Loss: 0.6350, Acc: 0.7500
Step [1900/20000], Loss: 0.5995, Acc: 0.6250
Step [2000/20000], Loss: 0.6391, Acc: 0.7500
Step [2100/20000], Loss: 0.6021, Acc: 0.6250
Step [2200/20000], Loss: 0.5877, Acc: 0.7500
Step [2300/20000], 

Step [18200/20000], Loss: 0.1904, Acc: 0.9375
Step [18300/20000], Loss: 0.2392, Acc: 0.8750
Step [18400/20000], Loss: 0.3504, Acc: 0.8750
Step [18500/20000], Loss: 0.2008, Acc: 1.0000
Step [18600/20000], Loss: 0.4306, Acc: 0.8125
Step [18700/20000], Loss: 0.2697, Acc: 0.8750
Step [18800/20000], Loss: 0.1381, Acc: 0.9375
Step [18900/20000], Loss: 0.4499, Acc: 0.8125
Step [19000/20000], Loss: 0.2715, Acc: 0.8125
Step [19100/20000], Loss: 0.2789, Acc: 0.8125
Step [19200/20000], Loss: 0.1786, Acc: 0.9375
Step [19300/20000], Loss: 0.1474, Acc: 1.0000
Step [19400/20000], Loss: 0.3025, Acc: 0.8125
Step [19500/20000], Loss: 0.3004, Acc: 0.8750
Step [19600/20000], Loss: 0.5279, Acc: 0.6875
Step [19700/20000], Loss: 0.3224, Acc: 0.9375
Step [19800/20000], Loss: 0.1828, Acc: 0.9375
Step [19900/20000], Loss: 0.4606, Acc: 0.7500
Step [20000/20000], Loss: 0.2107, Acc: 0.9375


In [7]:
correct = 0
total = 0
for images, labels in dataloader:
    images = Variable(images.cuda())
#     net.eval()
    outputs = cnn(images)
#     print(outputs.data)
    _, predicted = torch.max(outputs.data, 1)
#     print(predicted)
    total += labels.size(0)
    correct += (predicted.cpu() == labels).sum()
    
print('Accuracy of the network on the train images: %d %%' % (100*correct/total))

Accuracy of the network on the train images: 87 %


In [8]:
torch.save(cnn.state_dict(),'model_vgg_bn.pkl')

In [5]:
learning_rate = 0.00001
total_step = 10000
net = VGG_16()
net.cuda()
net.load_state_dict(torch.load('model_vgg_bn.pkl'))
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
logger = Logger('./logs_vgg_bn2')

data_iter = iter(dataloader)
iter_per_epoch = len(dataloader)

In [6]:
for step in range(total_step):
    if (step+1)%iter_per_epoch:
        data_iter = iter(dataloader)
    images, labels = next(data_iter)
    images, labels = Variable(images).cuda(), Variable(labels).cuda()
    optimizer.zero_grad()
    outputs = net(images)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    
    _, argmax = torch.max(outputs, 1)
    accuracy = (labels == argmax.squeeze()).float().mean()
    if (step+1) % 100 == 0:
        print ('Step [%d/%d], Loss: %.4f, Acc: %.4f' 
               %(step+1, total_step, loss.data[0], accuracy.data[0]))

        #============ TensorBoard logging ============#
        # (1) Log the scalar values
        info = {
            'loss': loss.data[0],
            'accuracy': accuracy.data[0]
        }

        for tag, value in info.items():
            logger.scalar_summary(tag, value, step+1)

Step [100/10000], Loss: 0.3241, Acc: 0.8125
Step [200/10000], Loss: 0.2328, Acc: 0.9375
Step [300/10000], Loss: 0.1594, Acc: 0.9375
Step [400/10000], Loss: 0.1974, Acc: 0.8750
Step [500/10000], Loss: 0.4362, Acc: 0.6875
Step [600/10000], Loss: 0.1052, Acc: 1.0000
Step [700/10000], Loss: 0.4611, Acc: 0.6875
Step [800/10000], Loss: 0.2384, Acc: 0.8750
Step [900/10000], Loss: 0.4951, Acc: 0.6875
Step [1000/10000], Loss: 0.3143, Acc: 0.7500
Step [1100/10000], Loss: 0.1014, Acc: 1.0000
Step [1200/10000], Loss: 0.3505, Acc: 0.9375
Step [1300/10000], Loss: 0.2088, Acc: 0.9375
Step [1400/10000], Loss: 0.4597, Acc: 0.7500
Step [1500/10000], Loss: 0.1962, Acc: 0.9375
Step [1600/10000], Loss: 0.1881, Acc: 0.9375
Step [1700/10000], Loss: 0.0845, Acc: 1.0000
Step [1800/10000], Loss: 0.3110, Acc: 0.9375
Step [1900/10000], Loss: 0.2963, Acc: 0.7500
Step [2000/10000], Loss: 0.4167, Acc: 0.8750
Step [2100/10000], Loss: 0.1105, Acc: 1.0000
Step [2200/10000], Loss: 0.1877, Acc: 0.9375
Step [2300/10000], 

In [8]:
torch.save(net.state_dict(),'model_vgg_bn2.pkl')

In [12]:
learning_rate = 0.00001
total_step = 10000
net2 = VGG_16()
net2.cuda()
net2.load_state_dict(torch.load('model_vgg_bn2.pkl'))
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
logger = Logger('./logs_vgg_bn3')

data_iter = iter(dataloader)
iter_per_epoch = len(dataloader)

In [13]:
for step in range(total_step):
    if (step+1)%iter_per_epoch:
        data_iter = iter(dataloader)
    images, labels = next(data_iter)
    images, labels = Variable(images).cuda(), Variable(labels).cuda()
    optimizer.zero_grad()
    outputs = net2(images)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    
    _, argmax = torch.max(outputs, 1)
    accuracy = (labels == argmax.squeeze()).float().mean()
    if (step+1) % 100 == 0:
        print ('Step [%d/%d], Loss: %.4f, Acc: %.4f' 
               %(step+1, total_step, loss.data[0], accuracy.data[0]))

        #============ TensorBoard logging ============#
        # (1) Log the scalar values
        info = {
            'loss': loss.data[0],
            'accuracy': accuracy.data[0]
        }

        for tag, value in info.items():
            logger.scalar_summary(tag, value, step+1)

Step [100/10000], Loss: 0.2127, Acc: 0.8750
Step [200/10000], Loss: 0.2975, Acc: 0.9375
Step [300/10000], Loss: 0.3010, Acc: 0.8750
Step [400/10000], Loss: 0.4482, Acc: 0.7500
Step [500/10000], Loss: 0.1947, Acc: 0.9375
Step [600/10000], Loss: 0.0754, Acc: 1.0000
Step [700/10000], Loss: 0.3919, Acc: 0.8125
Step [800/10000], Loss: 0.2257, Acc: 0.9375
Step [900/10000], Loss: 0.5379, Acc: 0.7500
Step [1000/10000], Loss: 0.2906, Acc: 0.8125
Step [1100/10000], Loss: 0.5314, Acc: 0.8125
Step [1200/10000], Loss: 0.2847, Acc: 0.8125
Step [1300/10000], Loss: 0.1318, Acc: 0.9375
Step [1400/10000], Loss: 0.1903, Acc: 0.8750
Step [1500/10000], Loss: 0.2306, Acc: 0.8750
Step [1600/10000], Loss: 0.2831, Acc: 0.9375
Step [1700/10000], Loss: 0.1910, Acc: 0.9375
Step [1800/10000], Loss: 0.1961, Acc: 0.8750
Step [1900/10000], Loss: 0.2632, Acc: 0.8750
Step [2000/10000], Loss: 0.4905, Acc: 0.8125
Step [2100/10000], Loss: 0.1666, Acc: 0.9375
Step [2200/10000], Loss: 0.0645, Acc: 1.0000
Step [2300/10000], 

In [14]:
correct = 0
total = 0
for images, labels in dataloader:
    images = Variable(images.cuda())
#     net2.eval()
    outputs = net2(images)
#     print(outputs.data)
    _, predicted = torch.max(outputs.data, 1)
#     print(predicted)
    total += labels.size(0)
    correct += (predicted.cpu() == labels).sum()
    
print('Accuracy of the network on the train images: %d %%' % (100*correct/total))

Accuracy of the network on the train images: 89 %


In [15]:
torch.save(net2.state_dict(),'model_vgg_bn3.pkl')